In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
%cd '/gdrive/My Drive'

/gdrive/My Drive


In [0]:
!git clone https://github.com/aitorzip/PyTorch-CycleGAN.git

Cloning into 'PyTorch-CycleGAN'...
remote: Enumerating objects: 60, done.
remote: Total 60 (delta 0), reused 0 (delta 0), pack-reused 60
Unpacking objects: 100% (60/60), done.


In [0]:
%cd '/gdrive/My Drive/PyTorch-CycleGAN'

/gdrive/My Drive/PyTorch-CycleGAN


In [0]:
%%sh
sh ./download_dataset summer2winter_yosemite

Archive:  ./datasets/summer2winter_yosemite.zip
   creating: ./datasets/summer2winter_yosemite/
   creating: ./datasets/summer2winter_yosemite/trainA/
  inflating: ./datasets/summer2winter_yosemite/trainA/2014-07-01 10:31:48.jpg  
  inflating: ./datasets/summer2winter_yosemite/trainA/2015-07-01 23:43:46.jpg  
  inflating: ./datasets/summer2winter_yosemite/trainA/2014-07-27 21:50:52.jpg  
  inflating: ./datasets/summer2winter_yosemite/trainA/2016-09-12 21:34:08.jpg  
  inflating: ./datasets/summer2winter_yosemite/trainA/2015-06-10 23:35:46.jpg  
  inflating: ./datasets/summer2winter_yosemite/trainA/2016-07-12 13:57:26.jpg  
  inflating: ./datasets/summer2winter_yosemite/trainA/2014-05-13 02:35:47.jpg  
  inflating: ./datasets/summer2winter_yosemite/trainA/2015-07-12 20:23:26.jpg  
  inflating: ./datasets/summer2winter_yosemite/trainA/2015-08-26 00:23:12.jpg  
  inflating: ./datasets/summer2winter_yosemite/trainA/2011-05-19 11:26:37.jpg  
  inflating: ./datasets/summer2winter_yosemite/tr

./download_dataset: 5: ./download_dataset: [[: not found
for details.

--2019-09-26 16:33:00--  https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/summer2winter_yosemite.zip
Resolving people.eecs.berkeley.edu (people.eecs.berkeley.edu)... 128.32.189.73
Connecting to people.eecs.berkeley.edu (people.eecs.berkeley.edu)|128.32.189.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 132640856 (126M) [application/zip]
Saving to: ‘./datasets/summer2winter_yosemite.zip’

     0K .......... .......... .......... .......... ..........  0%  180K 11m59s
    50K .......... .......... .......... .......... ..........  0% 32.9M 6m1s
   100K .......... .......... .......... .......... ..........  0%  370K 5m57s
   150K .......... .......... .......... .......... ..........  0% 29.3M 4m29s
   200K .......... .......... .......... .......... ..........  0% 33.5M 3m36s
   250K .......... .......... .......... .......... ..........  0%  370K 3m58s
   300K ..........

In [0]:
!mv datasets/summer2winter_yosemite /gdrive/My\ Drive/dl-pytorch/datasets/

In [0]:
!ls /gdrive/My\ Drive/dl-pytorch/datasets/

64x64_SIGNS	 cifar-10-batches-py	 summer2winter_yosemite
64x64_SIGNS.zip  cifar-10-python.tar.gz


In [0]:
import torch.nn as nn
import torch.nn.functional as F

class ResidualBlock(nn.Module):
  def __init__(self, in_features):
    super(ResidualBlock, self).__init__()
    
    conv_block = [ nn.ReflectionPad2d(1), #mejor padding
                   nn.Conv2d(in_features, in_features, 3),
                   nn.InstanceNorm2d(in_features), #BN para GANS
                   nn.ReLU(True),
                   nn.ReflectionPad2d(1), #mejor para consrvar distribucion
                   nn.Conv2d(in_features, in_features, 3),
                   nn.InstanceNorm2d(in_features)
                 ]
    
    self.conv_block = nn.Sequential(*conv_block)
    def forward(self, x):
      return self.conv_block(x) + x #una idea poderosa


In [0]:
class Generator(nn.Module):
  def __init__(self, input_nc, output_nc, n_residual_blocks=9):
    super(Generator,self).__init__()
    
    # Bloqueconvolucional
    model = [ nn.ReflectionPad2d(3),
            nn.Conv2d(input_nc, 64, F), # I - 7 + 6 /1 +1 = I
            nn.InstanceNorm2d(64),
             nn.ReLU(True)
            ]
    
    in_features = 64
    out_features = in_features * 2
    
    #Encoding
    for _ in range(2):
      model += [ nn.Conv2d(in_features, out_features, 3, stride=2, padding=1), #I/2
                 nn.InstanceNorm2d(out_features),
                 nn.ReLU(True)
               ]
      in_features = out_features
      out_features = in_features*2